<div>
    <img src="https://github.com/ignaciomsarmiento/IAMD/blob/main/Banner.jpg?raw=1" />
</div>



# Sistemas de Recomendación basado en Contenidos

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ignaciomsarmiento/IAMD/blob/main/NLP/Intro_to_Content_NLP.ipynb)


Este *cuaderno* trata sobre filtrado colaborativo basado en contenidos. El objetivo del *cuaderno* es que usted obtenga una visión general del problema predictivo de los sistemas de recomendación que utilizan filtrado colaborativo basado en contenidos, aprenda distintos algoritmos que lo implementan, y que sea capaz de reconocer sus características, funcionamiento, y  desarrollarlos en `Python`.


## Introducción

Los sistemas basados en contenido, a diferencia de los filtros colaborativos, no requieren datos relacionados a otros individuos u actividades pasadas. Por el contrario, estos brindan recomendaciones basadas en el perfil del usuario y los metadatos que se tiene sobre elementos particulares.

Si bien en este tipo de sistemas se utilizan la interacción entre un usuario y un ítem particular (lo compró, calificó, etc.) nosotros nos centraremos en el uso de texto asociado a los ítems. En este cuaderno en particular se abordará la construcción de dos tipos de recomendadores basados en contenido de películas, pero que pueden ser aplicados a otros productos:

 1. Recomendador basado en la descripción de la trama: este modelo compara las descripciones de diferentes películas y proporciona recomendaciones basado en películas con tramas similares.
 2. Recomendador basado en metadatos: este modelo tiene en cuenta una gran cantidad de características, como géneros, palabras clave, elenco, director, etc. A partir de ellos proporciona recomendaciones que son las más similares basadas en estas características.


## Texto como datos

Para poder utilizar el texto asociado a los ítems debemos primero transformarlo en datos que puedan ser utilizado por los sistemas de recomendación.

### Limpieza de datos

Antes de poder utilizar el texto necesitamos transformarlo de forma tal que las maquinas puedan utilizarlo. Hay al menos 3 pasos previos:

1. **Limpieza de texto** con **expresiones regulares (regex)** juegan un papel importante al automatizar tareas específicas de limpieza.

2. **Tokenización.**

3. **Lematización/stemmización.**



## Patrones básicos y Expresiones Regulares

Para ilustrar a qué nos referimos, comencemos con un ejemplo sencillo. El patrón más sencillo que se puede utilizar con expresiones regulares es utilizar secuencia de caracteres que uno quiere encontrar en el texto. Por ejemplo, si quisiéramos buscar la palabra *tienda* en un texto, simplemente podríamos usar como patrón `tienda`. Los patrones de búsqueda pueden estar conformados por un solo caracter como `!` para buscar signos de exclamación o también una secuencia de letras:


<div> <center>

| **RE** |      **Ejemplo del patrón capturado**     |
|:------:|:-----------------------------------------:|
| tienda | El que tenga <u>tienda</u> que la atienda |
|    a   |    El que m<u>a</u>druga Dios le ayuda    |
|    !   |            ¡Ojo con eso<u>!</u>           |

</center> </div>   

Este tipo de búsquedas es sensible al uso de mayúsculas, por ejemplo, buscar la palabra `tienda` arroja un resultado diferente al de buscar `Tienda`. Del mismo modo, también es sensible al uso de caracteres especiales como tildes, apostrofes, etc. En la práctica se suelen eliminar estos caracteres especiales para simplificar el texto analizado. Por ejemplo, transformar un texto como:

**<center> A palabras necias oídos sordos </center>**

por

**<center> a palabras necias oidos sordos </center>**

hará más sencillo su tratamiento. No obstante, las expresiones regulares son una herramienta superpoderosa y nos permiten usar funciones que simplifican la tarea. Por ejemplo, podemos usar los corchetes (`[]`) para expresar disyunción lógica (`o`). Por ejemplo, la búsqueda `[Tt]ienda` sirve para encontrar la palabra `tienda` **o** la palabra `Tienda`. Los corchetes indican que se busca una palabra que contenga la cadena `ienda` precedida por una letra `t` en minúscula **o** mayúscula. Por ejemplo:

<div> <center>

|    **RE**    |**Patrón capturado**|          **Ejemplo del patrón capturado**            |
|:------------:|:----------------:|:------------------------------------------------------:|
|   [Tt]ienda  |  Tienda o tienda |        El que tiene <u>tienda</u> que la atienda       |
|     [abc]    |   a, b **o** c   | No me <u>a</u>bra los ojos que no le voy a echar gotas |
| [1234567890] | Cualquier dígito |            Eramos entre <u>5</u> y 8 personas   |

</div> </center>


Notemos en la última linea que la expresión regular `[1234567890]` nos permite capturar cualquier dígito, no obstante, escribir bloques de dígitos o letras puede ser inconveniente. Es decir, para capturar cualquier letra no es práctico escribir todo el abecedario: `[abcdefghijklmnopqrstuvwxyz]`. En estos casos uno puede completar la búsqueda dentro de corchetes con un guión (`-`) que especifica rangos. Por ejemplo `[0-9]` nos permite capturar cualquier número entre 0 y 9, `[b-g]` nos permite capturar cualquier letra de la `b` a la `g` o sea *b, c, d, e, f **o** g*.

<div> <center>

| **RE** |     **Patrón capturado**     |            **Ejemplo del patrón capturado**            |
|:------:|:----------------------------:|:------------------------------------------------------:|
|  [A-Z] | Cualquier letra en mayúscula |        <u>E</u>l que tiene tienda que la atienda       |
|  [a-z] | Cualquier letra en minúscula | N<u>o</u> me abra los ojos que no le voy a echar gotas |
|  [0-9] |       Cualquier dígito       |       Eramos entre <u>5</u> y 8 personas       |

</div> </center>

Podemos también indicar que caracteres no deben ser capturados, para ello utilizamos un caret (`^`) al inicio del corchete `[^]`. Sólo si el caret (`^`) es el primer símbolo dentro del corchete, el patrón subsiguiente es negado. Por ejemplo, `[^a]` significa que se va a capturar cualquier caracter, incluyendo los especiales, excepto la letra *a*.


<div> <center>

| **RE** |               **Patrón capturado**              |            **Ejemplo del patrón capturado**            |
|:------:|:-----------------------------------------------:|:------------------------------------------------------:|
| [^A-Z] | Cualquier caracter menos una letra en mayúscula |       E<u>l</u> que tiene tienda que la atienda       |
|  [^Ss] |     Cualquier caracter excepto una "s" o "S"    | <u>N</u>o me abra los ojos que no le voy a echar gotas |
|  [^.]  |        Cualquier caracter menos un punto        |       <u>E</u>ramos al rededor de 5 a 8 personas       |
|  [e^]  |             Captura una "e" o un "^"            |                       <u>e</u>^x                       |
|  [a^b] |                   Captura a^b                   |                 La expresión <u>a^b</u>                |

</div> </center>


Note sin embargo, que si se usa el caret (`^`) en cualquier otro lugar de la expresión regular, este no va a significar una negación, sino un caret.

Del mismo modo, a menudo buscamos capturar patrones opcionales. Por ejemplo, para capturar una palabra en plural o en singular en donde el último caracter es una *s*. Para esto utilizamos el símbolo de pregunta (`?`) después del caracter opcional. El signo de pregunta (`?`) en el contexto de expresiones regulares significa el caracter anterior o ninguno.


<div> <center>

|  **RE**  | **Patrón capturado** |            **Ejemplo del patrón capturado**           |
|:--------:|:--------------------:|:-----------------------------------------------------:|
| tiendas? | "tienda" o "tiendas" |       El que tenga <u>tienda</u> que la atienda       |
|  colou?r |  "color" o "colour"  | Discover the newest hand-picked <u>color</u> palettes |

</div> </center>


Pero también existen casos donde un caracter se puede repetir más de una vez. Por ejemplo, en un libro se podría encontrar la onomatopeya del mujido de una vaca de diversas formas:

**<center> Muu! </center>**

**<center> Muuu! </center>**

**<center> Muuuu! </center>**

**<center> Muuuuu! </center>**

A grandes rasgos, podemos describir esta onomatopeya como una palabra que comienza con una *M* seguida con por lo menos dos *u* y finaliza con el signo de exclamación *!*. La expresión regular que nos permite capturar cero o más ocurrencias de un caracter es el asterisco (`*`) también conocido como *cleany star* o *Kleene* . Por ende, la expresión regular `u*` va a capturar tanto `u` como `uuuuuu`, pero a su vez también podría capturar `vaca` pues esta palabra tiene cero letras u.

Para corregir esto, podríamos usar la expresión regular `uu*` la cual significa una o más letras u. Algunos patrones más complejos también se pueden utilizar haciendo uso de los corchetes; por ejemplo `[ab]\*` sirve para capturar cero o más *a*s o *b*s. Por ende, se capturarían textos como *aaaaa*, *bbb* o *ababababab*.

Asimismo, para especificar múltiples dígitos podemos usar `[0-9][0-9]*` para capturar cualquier entero.

Sin embargo, aún podemos utilizar el signo de suma (`+`), también llamado *Kleene +*, para simplificar las expresiones regulares. El *Kleene +*, nos permite denotar que el caracter a capturar se repite una o más veces. Por ende, la expresión `[0-9]+` es la forma más común de expresar una secuencia de dígitos. Por ejemplo:

<div> <center>

|  **RE**  | **Patrón capturado** |            **Ejemplo del patrón capturado**           |
|:--------:|:--------------------:|:-----------------------------------------------------:|
   |
|   baa*   |  ba con una o más as |               La cabra hace <u>baaa</u>!              |
    |   mu+!   | mu! con una o más us |              La vaca hizo <u>muuuuu!</u>           
|  [0-9]+  |   Cualquier entero   |              Ese camisa cuesta $<u>25</u>             |

</div> </center>


Otra función importante esta dada por el punto (`.`). Este funciona como comodín o *wildcard*. Esta expresión regular sirve para capturar cualquier caracter excepto los saltos de línea.


<div> <center>

|  **RE**  | **Patrón capturado** |            **Ejemplo del patrón capturado**           |
|:--------:|:--------------------:|:-----------------------------------------------------:|
   |
|   1. | 10 y 1A             | Ganaron el partido <u>18</u> a 2             |
|  1.4 | 114 y 1B4            | Vive en el apartamento <u>1B4<u> |

</div> </center>

También existen los denominado anclas o *anchors* que sirven para capturar elementos en posiciones particulares del texto. Los más comunes son el caret (`^`) y el símbolo de dolar (`$`) los cuales hacen alusión al inicio y final de un texto respectivamente. Por ejemplo, la expresión `^El` solo captura la palabra *El* sólo si está al inicio del corpus de texto. Otras anclas comunes son (`\b`) y (`\B`) que denotan los *boundaries* o límites de una palabra o dentro de una palabra respectivamente. Por ejemplo, `\bel\b` va a capturar la palabra *el* pero no *elefante*.


### Disyunción, agrupación y precedencia


A menudo estaremos interesados en buscar más de una palabra a la vez. Por ejemplo, si en nuestro texto quisiéramos buscar países de Latinoamérica,  escribir `[ColombiaPerúChileMéxico...]` sólo nos devolvería alguna de las letras presentes en los corchetes. El operador de disyunción (`|`) sirve para este tipo de casos donde estamos interesados en una u otra palabra, por eso, el patrón `Colombia|Perú|Chile` devuelve Colombia, Perú o Chile.

A su vez, puede que nos interesemos no sólo en los países como tal, sino también los gentilicios. Por ejemplo, si queremos extraer Chile o Chilenos necesitamos sofisticar nuestro operador de disyunción para evitar escribir la expresión `Chile|Chilenos`. En este caso podemos utilizar  paréntesis para inidcar que la disyunción sólo aplica para una parte del texto: `Chil(e|enos)`. Note que si omitiéramos los paréntesis `Chile|enos` solo se capturaría *Chile* o *enos* y dado que *Chile* tiene precedencia sobre *enos* en caso de encontrar la palabra *Chilenos* sólo se extraería la primer parte: *Chile* sin el *nos*.

Los paréntesis también son un gran complemento para los asteriscos (`*`). Supongamos poseemos el índice de un libro que tiene el siguiente texto: Capítulo 1, Capítulo 2, Capítulo 3, etc. Para capturar todos los capítulos necesitamos crear un patrón que capture repetidamente la palabra *Capítulo* seguida de algún entero. La expresión regular `Capítulo [0-9]+ *` sólo captura *Capítulo* seguida de un entero y 0 más espacios, en este caso necesitamos utilizar los paréntesis: `(Capítulo [0-9]+ *)*`.

### Algunos operadores adicionales

<div> <center>

| **RE** | **Expansión** |                          **Patrón capturado**                         |
|:------:|:-------------:|:---------------------------------------------------------------------:|
|   \d   |     [0-9]     |                            Cualquier dígito                           |
|   \D   |     [^0-9]    |                          Cualquier no dígito                          |
|   \w   |  [a-zA-Z0-9_] |                  Cualquier alfanumérico o guion bajo                  |
|   \W   |     [ˆ\w]     |                 Cualquier no alfanumérico o guion bajo                |
|   \s   |  [ \r\t\n\f]  |                           Espacio en blanco                           |
|   \S   |     [ˆ\s]     |                        No un espacio en blanco                        |
|    *   |               |         Cero o más ocurrencias del caracter o expresión pasada        |
|    +   |               |         Una o más ocurrencias del caracter o expresión pasada         |
|    ?   |               | Exactamente cero o una ocurrencia del del caracter o expresión pasada |
|   {n}  |               |            *n* ocurrencias del caracter o expresión pasada            |
|  {n,m} |               |        De *n* a *m* ocurrencias del caracter o expresión pasada       |
|  {n,}  |               |      Por lo menos *n* ocurrencias del caracter o expresión pasada     |
|  {,m}  |               |         Hasta *m* ocurrencias del caracter o expresión pasada         |

</div> </center>

Estos son algunos de los patrones y operadores básicos que vamos a necesitar y utilizar en este curso; sin embargo hay mucho más y los invito a explorarlos por su cuenta. Antes de ilustrar el uso en `Python` es importante recomendar la página https://regexr.com  que permite probar el correcto funcionamiento de las expresiones regulares creadas antes de utilizarlas en el código.

In [1]:
!pip install pdfplumber

  Using cached pdfplumber-0.11.8-py3-none-any.whl.metadata (43 kB)
  Using cached pdfminer_six-20251107-py3-none-any.whl.metadata (4.2 kB)
  Using cached pypdfium2-5.1.0-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (67 kB)
Using cached pdfplumber-0.11.8-py3-none-any.whl (60 kB)
Using cached pdfminer_six-20251107-py3-none-any.whl (5.6 MB)
Using cached pypdfium2-5.1.0-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)


In [2]:
# Cargamos pdfplumber
import pdfplumber
import requests
from io import BytesIO

# URL directa al PDF en GitHub
url = "https://github.com/ignaciomsarmiento/datasets/blob/main/telefonos.pdf?raw=true"

# Obtenemos el contenido del PDF usando requests
response = requests.get(url, stream=True)
response.raise_for_status()  # Raise an exception for bad responses (4xx or 5xx)

# Abrimos el PDF con pdfplumber usando BytesIO para manejar el contenido
with pdfplumber.open(BytesIO(response.content)) as pdf:
    paginas = pdf.pages
    documento = ""
    for pag in paginas:
        text = pag.extract_text()
        documento = documento + " " + text

# Visualizamos 500 caracteres del texto
print(documento)


 Boletín interno - Universidad Ficticia
Si tienes dudas sobre el curso de Programación:
- Profesor: Juan Pérez
- Correo: juan.perez@uni-ficticia.edu
- Teléfono: +57 312 456 7890
Coordinación académica:
- Correo: coordinacion.academica@uni-ficticia.edu.co
- Teléfono: (1) 765 4321
Soporte técnico:
- Correo: soporte-ti@uni-ficticia.edu
- Teléfono: 320-987-6543
Otros contactos:
- María López - maria_lopez93@gmail.com - Tel: 604 555 7788
- Pedro Gómez - pedro.gomez@empresa.com - Cel: +57-301-222-3344


Vamos a regex101.com y probamos la siguiente expresión regular para extraer los números de teléfono

In [ ]:
Ahora en Python

In [3]:
import re

patron = r"\+57 312 456 7890"
print(re.findall(patron, documento))

['+57 312 456 7890']


In [8]:
print(re.findall(r"\+57[\s+-]3\d{2}[\s+-]\d{3}[\s+-]\d{4}",documento))

['+57 312 456 7890', '+57-301-222-3344']


Cualquier patron

In [9]:

patron_telefonos = re.compile(r"""
    (?:\+57[\s-]?)?           # opcional: +57 con espacio o guion
    (?:\(?\d{1,3}\)?[\s-]?)?  # opcional: (1), 320-, 604 , etc.
    \d{3}[\s-]?\d{4}          # parte final: 3 dígitos, sep, 4 dígitos
""", re.VERBOSE)

coincidencias = patron_telefonos.findall(documento)
for tel in coincidencias:
    print("-", tel)


- +57 312 456 7890
- (1) 765 4321
- 320-987-6543
- 604 555 7788
- +57-301-222-3344


Los correos electrónicos

In [10]:

patron_email = r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9.-]+"

coincidencias = re.findall(patron_email, documento)
for mail in coincidencias:
    print("-", mail)


- juan.perez@uni-ficticia.edu
- coordinacion.academica@uni-ficticia.edu.co
- soporte-ti@uni-ficticia.edu
- maria_lopez93@gmail.com
- pedro.gomez@empresa.com



# Limpieza de comentarios de Amazon con expresiones regulares y spellchecker


1. Leer reseñas de Amazon mezclando español e inglés con errores reales.
2. Comprender qué hace cada regex: limpiar HTML, URLs, correos, símbolos, repeticiones.
3. Tokenizar y remover stopwords según idioma usando listas locales.
4. Aplicar un corrector ortográfico básico (`pyspellchecker`) y ver cómo cambia el texto.
5. Dejar espacio para que ajustes patrones y vocabularios propios según tu intuición.


## Dependencias

Usaremos `pyspellchecker` para corrección ortográfica. (Ejecutar una sola vez o saltar si ya están instaladas):

In [12]:
# Instalar dependencias si hace falta
!pip install pyspellchecker


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 82.0 MB/s eta 0:00:00


## Cargar datos de reseñas
- El archivo está en `data/amazon_reviews_sample.csv` con columnas `review_id`, `lang`, `rating`, `title`, `comment`.
- Hay 10 reseñas en español y 10 en inglés, con errores de tipeo y ruido (tildes omitidas, abreviaturas, repeticiones de letras, etc.).
- Ejecuta la celda para ver el `head()` y asegurarte de que la ruta funciona en tu entorno.


In [16]:
import pandas as pd
from pathlib import Path


df = pd.read_csv('https://raw.githubusercontent.com/ignaciomsarmiento/IAMD/refs/heads/main/NLP/data/amazon_reviews_sample.csv')
df.head()


,review_id,lang,rating,title,comment
0,r001,es,5,Excelente producto,"El envio llego rapidisimo, pero el empaque est..."
1,r002,es,3,Regular,Lo use dos semanas y ya empezo a fallar :( la ...
2,r003,es,4,Buena compra,"Me gusto, aunque las instruciones vienen solo ..."
3,r004,es,1,Mala calidad,"No sirvio, venia sucio y parecia reusado. Adem..."
4,r005,es,2,Decepcion,La pantalla tenia un pixel muerto y el vendedo...


## Patrones de limpieza con regex (paso a paso)


1. `lower()` + `strip_accents`: deja todo en minúscula y quita tildes para que las búsquedas no dependan de acentos.
2. `re.sub(r'<[^>]+>', ' ', text)`: elimina cualquier etiqueta HTML que pueda venir de un scrape.
3. `re.sub(r'https?://\S+|www\.\S+', ' ', text)`: borra URLs.
4. `re.sub(r'\S+@\S+', ' ', text)`: borra correos electrónicos.
5. `re.sub(r'[^a-z\s]', ' ', text)`: deja solo letras; elimina números y signos. Cambia el patrón si quieres conservar dígitos o `#`.
6. `re.sub(r'(.){2,}', r'', text)`: colapsa repeticiones largas ("bueeeno" → "bueeno"); ajusta `{2,}` si quieres ser más o menos agresivo.
7. `re.sub(r'\s+', ' ', ...)`: colapsa espacios múltiples y hace `strip()` al final.

Después tokenizamos con `re.findall(r"[a-zA-Z]+", text)` y quitamos stopwords específicas para `es` o `en`. Finalmente aplicamos spellchecker por idioma.


In [34]:
import re
import unicodedata
from typing import List
from spellchecker import SpellChecker

# Stopwords locales

df_stop_es = pd.read_csv('https://raw.githubusercontent.com/ignaciomsarmiento/IAMD/refs/heads/main/NLP/data/stopword_extend.csv')
print(df_stop_es)

    palabra
0         a
1         á
2       acá
3    ademas
4    además
..      ...
460       x
461       y
462      ya
463      yo
464       z

[465 rows x 1 columns]


In [35]:
import requests

url = "https://raw.githubusercontent.com/ignaciomsarmiento/IAMD/refs/heads/main/NLP/data/sw_DNP.txt"

try:
    response = requests.get(url)
    response.raise_for_status()  # Raises an HTTPError for bad responses (4xx or 5xx)
    print(response.text)
except requests.exceptions.RequestException as e:
    print(f"Error al obtener el archivo: {e}")

a
ademas
además
ahi
ahí
al
algo
alguna
algunas
alguno
algunos
algún
alli
allí
alrededor
ambos
ante
antes
aquel
aquella
aquellas
aquello
aquellos
aquél
aquélla
aquéllas
aquéllos
asi
así
aun
aunque
aún
b
c
como
con
contra
cual
cuales
cuando
cuál
cuáles
cómo
d
de
del
demás
desde
donde
durante
dónde
e
el
ella
ellas
ellos
en
entre
era
erais
eran
eras
eres
es
esa
esas
ese
eso
esos
esta
estaba
estabais
estaban
estabas
estad
estada
estadas
estado
estados
estamos
estando
estar
estaremos
estará
estarán
estarás
estaré
estaréis
estaría
estaríais
estaríamos
estarían
estarías
estas
este
estemos
esto
estos
estoy
estuve
estuviera
estuvierais
estuvieran
estuvieras
estuvieron
estuviese
estuvieseis
estuviesen
estuvieses
estuvimos
estuviste
estuvisteis
estuviéramos
estuviésemos
estuvo
está
estábamos
estáis
están
estás
esté
estéis
estén
estés
fue
fuera
fuerais
fueran
fueras
fueron
fuese
fueseis
fuesen
fueses
fui
fuimos
fuiste
fuisteis
fuéramos
fuésemos
ha
habida
habidas
habido
habidos
habiendo
habremos
hab

In [38]:
# En este pedazo de codigo definimos las funciones para limpiar nuestros comentarios.

spanish_stopwords = set(
    unicodedata.normalize('NFKD', w).encode('ascii', 'ignore').decode('ascii')
    for w in df_stop_es['palabra'].dropna().str.lower().unique()
)
english_stopwords = {
    'the','and','to','of','a','in','it','is','for','on','that','this','with','as','was','but','are','not','be','have','had','has','at','by','from','or','an','so','if','too','very','can','will','just','about','after','only','more','when','than','also','been','into','its','they','we','you','your','my','me','our','their','them','what','which','there','here','over','again','all','any','some','most','such','no','nor','off','up','down','out','who','how','why','where','does','did','doing','could','would','should','because','while'
}

lang_stopwords = {'es': spanish_stopwords, 'en': english_stopwords}

# Funciones de limpieza
def strip_accents(text: str) -> str:
    text = unicodedata.normalize('NFKD', text)
    return text.encode('ascii', 'ignore').decode('ascii')


def basic_regex_clean(text: str) -> str:
    text = str(text).lower()  # a minusculas
    text = strip_accents(text) # quita los acentos
    text = re.sub(r'<[^>]+>', ' ', text)               # etiquetas HTML
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text)  # URLs
    text = re.sub(r'\S+@\S+', ' ', text)             # correos
    text = re.sub(r'[^a-z\s]', ' ', text)             # todo lo que no sea letra
    text = re.sub(r'(.){1,}', r'', text)      # colapsar letras repetidas
    text = re.sub(r'\s+', ' ', text).strip() # saca los espacios en blanco
    return text


def tokenize(text: str) -> List[str]:
    return re.findall(r"[a-zA-Z]+", text) # 'genera las tokens, un token es la unidad minima de analisis, en nuestro caso, son las palabras


def remove_stopwords(tokens: List[str], lang: str) -> List[str]:
    stops = lang_stopwords.get(lang, set())
    return [tok for tok in tokens if tok not in stops]


def spellcheck_tokens(tokens: List[str], lang: str) -> List[str]:
    lang_code = 'es' if lang == 'es' else 'en'
    spell = SpellChecker(language=lang_code, distance=1)
    corregidos = []
    for tok in tokens:
        sugerido = spell.correction(tok)
        corregidos.append(sugerido if sugerido else tok)
    return corregidos


## Aplicar el pipeline


In [43]:
def limpiar_fila(row):
    limpio = basic_regex_clean(row['comment'])
    #tokens = tokenize(limpio)
    #tokens = remove_stopwords(tokens, row['lang'])
    #tokens = spellcheck_tokens(tokens, row['lang'])
    return {
        'clean_text': limpio,
        #'tokens': tokens,
        #'spellchecked': ' '.join(tokens),
    }

procesado = df.apply(limpiar_fila, axis=1, result_type='expand')
df_limpio = pd.concat([df, procesado], axis=1)
#df_limpio[['review_id','lang','rating','comment','spellchecked']].head(10)
df_limpio[['review_id','lang','rating','comment','clean_text']].head(10)


,review_id,lang,rating,comment,clean_text
0,r001,es,5,"El envio llego rapidisimo, pero el empaque est...",el envio llego rapidisimo pero el empaque esta...
1,r002,es,3,Lo use dos semanas y ya empezo a fallar :( la ...,lo use dos semanas y ya empezo a fallar la bat...
2,r003,es,4,"Me gusto, aunque las instruciones vienen solo ...",me gusto aunque las instruciones vienen solo e...
3,r004,es,1,"No sirvio, venia sucio y parecia reusado. Adem...",no sirvio venia sucio y parecia reusado ademas...
4,r005,es,2,La pantalla tenia un pixel muerto y el vendedo...,la pantalla tenia un pixel muerto y el vendedo...
5,r006,es,5,"Pequeno pero potente, el sonido es clarito. Lo...",pequeno pero potente el sonido es clarito lo c...
6,r007,es,4,"La camara es buena, pero el estabilizador no e...",la camara es buena pero el estabilizador no es...
7,r008,es,3,La caja llego golpeada y el cable fue distinto...,la caja llego golpeada y el cable fue distinto...
8,r009,es,5,"Entrega antes de tiempo, todo bn empacado. Lo ...",entrega antes de tiempo todo bn empacado lo us...
9,r010,es,2,"Traia olor raro, y al segundo dia ya no encend...",traia olor raro y al segundo dia ya no encendi...


from matplotlib import pyplot as plt
_df_0['rating'].plot(kind='hist', bins=20, title='rating')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['rating'].plot(kind='line', figsize=(8, 4), title='rating')
plt.gca().spines[['top', 'right']].set_visible(False)

In [44]:
df_limpio

,review_id,lang,rating,title,comment,clean_text
0,r001,es,5,Excelente producto,"El envio llego rapidisimo, pero el empaque est...",el envio llego rapidisimo pero el empaque esta...
1,r002,es,3,Regular,Lo use dos semanas y ya empezo a fallar :( la ...,lo use dos semanas y ya empezo a fallar la bat...
2,r003,es,4,Buena compra,"Me gusto, aunque las instruciones vienen solo ...",me gusto aunque las instruciones vienen solo e...
3,r004,es,1,Mala calidad,"No sirvio, venia sucio y parecia reusado. Adem...",no sirvio venia sucio y parecia reusado ademas...
4,r005,es,2,Decepcion,La pantalla tenia un pixel muerto y el vendedo...,la pantalla tenia un pixel muerto y el vendedo...
5,r006,es,5,Me encanto,"Pequeno pero potente, el sonido es clarito. Lo...",pequeno pero potente el sonido es clarito lo c...
6,r007,es,4,Cumple,"La camara es buena, pero el estabilizador no e...",la camara es buena pero el estabilizador no es...
7,r008,es,3,Esperaba mas,La caja llego golpeada y el cable fue distinto...,la caja llego golpeada y el cable fue distinto...
8,r009,es,5,Rapido,"Entrega antes de tiempo, todo bn empacado. Lo ...",entrega antes de tiempo todo bn empacado lo us...
9,r010,es,2,No funciono,"Traia olor raro, y al segundo dia ya no encend...",traia olor raro y al segundo dia ya no encendi...
